# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-05-29 20:48:06.379836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 20:48:06.955398: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-29 20:48:07.068829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 20:48:07.068847: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQLBAV7/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQLBAV7/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-05-29 20:48:53.605173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 20:48:53.605303: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 20:48:53.605353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-29 20:48:53.605389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-29 20:48:53.605423: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting from the one on the left were the embedding is learned in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the kernel to move _only_ in the word by word direction (The alternative, moving coordinate by coordinate of the embedding space doesn't make much sense).

So let's create a model that use convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [7]:
# YOUR CODE HERE
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tk = Tokenizer()
tk.fit_on_texts(X_train)

X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype="float32", padding="post", maxlen=150)
X_test_pad = pad_sequences(X_test_token, dtype="float32", padding="post", maxlen=150)

In [20]:
X_train_pad[1].size

150

In [11]:
print(vocab_size := len(tk.word_index))

30419


## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel is taking into account ;)

In [32]:
# YOUR CODE HERE
from tensorflow.keras import layers, Sequential

model = Sequential([

layers.Embedding(input_dim=vocab_size+1, output_dim=150, mask_zero=True, input_length=150),
layers.Conv1D(20, kernel_size=3),
layers.Flatten(),
layers.Dense(10),
layers.Dense(1, activation="sigmoid")
    
])

model.compile(loss="binary_crossentropy",
             optimizer="rmsprop",
             metrics="accuracy")

❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercise (esp. the first one)

In [33]:
# YOUR CODE HERE
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 150, 150)          4563000   
                                                                 
 conv1d_5 (Conv1D)           (None, 148, 20)           9020      
                                                                 
 flatten_5 (Flatten)         (None, 2960)              0         
                                                                 
 dense_5 (Dense)             (None, 1)                 2961      
                                                                 
Total params: 4,574,981
Trainable params: 4,574,981
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [35]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 20ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.4812 - val_accuracy: 0.8120
Epoch 2/20
55/55 [==============================] - 1s 20ms/step - loss: 1.6508e-04 - accuracy: 1.0000 - val_loss: 0.6832 - val_accuracy: 0.7867
Epoch 3/20
55/55 [==============================] - 1s 20ms/step - loss: 3.6062e-04 - accuracy: 1.0000 - val_loss: 0.5594 - val_accuracy: 0.8040
Epoch 4/20
55/55 [==============================] - 1s 20ms/step - loss: 1.6673e-05 - accuracy: 1.0000 - val_loss: 0.6210 - val_accuracy: 0.8053
Epoch 5/20
55/55 [==============================] - 1s 21ms/step - loss: 2.3698e-06 - accuracy: 1.0000 - val_loss: 0.6847 - val_accuracy: 0.8067
Epoch 6/20
55/55 [==============================] - 1s 20ms/step - loss: 5.6297e-07 - accuracy: 1.0000 - val_loss: 0.7447 - val_accuracy: 0.8120
The accuracy evaluated on the test set is of 79.240%


# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you for real world challenges - but you have already done all the building bricks in the previous exercises

In [36]:
# YOUR CODE HERE
import gensim.downloader as api

print(list(api.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [38]:
word2vec_tl = api.load("glove-wiki-gigaword-50")

print(len(word2vec_tl.key_to_index))
print(len(word2vec_tl["dog"]))

400000
50


In [43]:
# Convert a sentence into a matrix representing the words in the embedding
def embed_sentences_tl(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
    
    return np.array(embedded_sentence)

In [44]:
# Convert a list of sentences into a list of matrices

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentences = embed_sentences_tl(word2vec, sentence)
        embed.append(embedded_sentences)
    
    return embed

In [45]:
X_train_embed = embedding(word2vec_tl, X_train)
X_test_embed = embedding(word2vec_tl, X_test)

In [46]:
X_train_pad = pad_sequences(X_train_embed, dtype="float32", padding="post", maxlen=150)
X_test_pad = pad_sequences(X_test_embed, dtype="float32", padding="post", maxlen=150)

❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer, and an output layer. Compile it, and fit it on the train data. You can then evaluate it on the test set.

In [49]:
# YOUR CODE HERE
model = Sequential([
    layers.Conv1D(20, kernel_size=3),
    layers.Flatten(),
    layers.Dense(10),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",
             optimizer="rmsprop",
             metrics="accuracy")

In [50]:
es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad,
         y_train,
         validation_split=0.3,
         epochs=50,
         batch_size=24,
         verbose=1,
         callbacks=[es])

Epoch 1/50
73/73 [==============================] - 1s 9ms/step - loss: 0.9686 - accuracy: 0.5383 - val_loss: 0.7009 - val_accuracy: 0.6080
Epoch 2/50
73/73 [==============================] - 0s 7ms/step - loss: 0.5309 - accuracy: 0.7280 - val_loss: 0.7338 - val_accuracy: 0.6360
Epoch 3/50
73/73 [==============================] - 0s 6ms/step - loss: 0.3626 - accuracy: 0.8429 - val_loss: 0.8598 - val_accuracy: 0.6253
Epoch 4/50
73/73 [==============================] - 0s 6ms/step - loss: 0.2473 - accuracy: 0.9097 - val_loss: 0.9502 - val_accuracy: 0.6253
Epoch 5/50
73/73 [==============================] - 0s 6ms/step - loss: 0.1587 - accuracy: 0.9480 - val_loss: 1.1365 - val_accuracy: 0.6213


In [52]:
print("Accuracy Score: ", model.evaluate(X_test_pad, y_test)[1])

79/79 [==============================] - 0s 3ms/step - loss: 0.7220 - accuracy: 0.5928
Accuracy Score:  0.5928000211715698


❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)